In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!git clone https://github.com/resemble-ai/Resemblyzer.git
!cd Resemblyzer

Cloning into 'Resemblyzer'...
remote: Enumerating objects: 624, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 624 (delta 69), reused 60 (delta 59), pack-reused 542
Receiving objects: 100% (624/624), 101.46 MiB | 38.34 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [2]:
!pip install resemblyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73461 sha256=1268c0212593e8415cba8fd16e1da0bed5eef95b00272f23c3baf4fbb0d18805
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26305 sha256=931b767a915355b5ea7e336c5a811ee91adb6981f49cce4ad6497ed40d279786
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built webrtcvad typing


In [3]:
from resemblyzer import preprocess_wav,VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = '/content/drive/MyDrive/datasets/speech_diarization/Audio/meeting-clip1.mp3'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.04 seconds.
(822, 256)


In [4]:
!pip3 install spectralcluster

In [5]:
from spectralcluster import RefinementOptions
from spectralcluster import ThresholdType
from spectralcluster import ICASSP2018_REFINEMENT_SEQUENCE

refinement_options = RefinementOptions(
    gaussian_blur_sigma=1,
    p_percentile=0.90,
    )

In [6]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    refinement_options=refinement_options
    )

labels = clusterer.predict(cont_embeds)

In [7]:
def create_labelling(labels,wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling

labelling = create_labelling(labels,wav_splits)

In [8]:
labelling

[('1', 0, 3.62),
 ('0', 3.62, 9.68),
 ('1', 9.68, 12.02),
 ('0', 12.02, 12.14),
 ('1', 12.14, 12.2),
 ('0', 12.2, 15.44),
 ('1', 15.44, 15.56),
 ('0', 15.56, 15.62),
 ('1', 15.62, 17.42),
 ('0', 17.42, 18.08),
 ('1', 18.08, 18.14),
 ('0', 18.14, 18.2),
 ('1', 18.2, 18.32),
 ('0', 18.32, 18.38),
 ('1', 18.38, 19.88),
 ('0', 19.88, 22.52),
 ('1', 22.52, 24.38),
 ('0', 24.38, 25.04),
 ('1', 25.04, 25.7),
 ('0', 25.7, 25.76),
 ('1', 25.76, 25.88),
 ('0', 25.88, 26.06),
 ('1', 26.06, 26.12),
 ('0', 26.12, 28.34),
 ('1', 28.34, 28.58),
 ('0', 28.58, 28.7),
 ('1', 28.7, 28.76),
 ('0', 28.76, 34.34),
 ('1', 34.34, 34.7),
 ('0', 34.7, 34.76),
 ('1', 34.76, 34.82),
 ('0', 34.82, 35.54),
 ('1', 35.54, 35.84),
 ('0', 35.84, 35.96),
 ('1', 35.96, 36.02),
 ('0', 36.02, 37.46),
 ('1', 37.46, 39.02),
 ('0', 39.02, 40.28),
 ('1', 40.28, 41.48),
 ('0', 41.48, 42.08),
 ('1', 42.08, 42.14),
 ('0', 42.14, 42.92),
 ('1', 42.92, 44.6),
 ('0', 44.6, 45.62),
 ('1', 45.62, 45.92),
 ('0', 45.92, 46.22),
 ('1', 4

In [9]:
for label in labelling:
  print(f"speaker {label[0]} spoke from {label[1]} seconds to {label[2]} seconds")

speaker 1 spoke from 0 seconds to 3.62 seconds
speaker 0 spoke from 3.62 seconds to 9.68 seconds
speaker 1 spoke from 9.68 seconds to 12.02 seconds
speaker 0 spoke from 12.02 seconds to 12.14 seconds
speaker 1 spoke from 12.14 seconds to 12.2 seconds
speaker 0 spoke from 12.2 seconds to 15.44 seconds
speaker 1 spoke from 15.44 seconds to 15.56 seconds
speaker 0 spoke from 15.56 seconds to 15.62 seconds
speaker 1 spoke from 15.62 seconds to 17.42 seconds
speaker 0 spoke from 17.42 seconds to 18.08 seconds
speaker 1 spoke from 18.08 seconds to 18.14 seconds
speaker 0 spoke from 18.14 seconds to 18.2 seconds
speaker 1 spoke from 18.2 seconds to 18.32 seconds
speaker 0 spoke from 18.32 seconds to 18.38 seconds
speaker 1 spoke from 18.38 seconds to 19.88 seconds
speaker 0 spoke from 19.88 seconds to 22.52 seconds
speaker 1 spoke from 22.52 seconds to 24.38 seconds
speaker 0 spoke from 24.38 seconds to 25.04 seconds
speaker 1 spoke from 25.04 seconds to 25.7 seconds
speaker 0 spoke from 25.7